### Importing libraries

In [268]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import functions
from pprint import pprint
%matplotlib inline

In [269]:
#df_csv = pd.read_csv("analytic_data2019.csv",skiprows=[1])
xls = pd.ExcelFile('2019 County Health Rankings Data - v2.xls')
df1 = pd.read_excel(xls, 'Additional Measure Data',skiprows=1)

In [270]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3073 entries, 0 to 3141
Data columns (total 78 columns):
FIPS                                        3073 non-null int64
State                                       3073 non-null object
County                                      3073 non-null object
Life_Expectancy                             3073 non-null float64
Life_Expectancy_(Black)                     1339 non-null float64
Life_Expectancy_(Hispanic)                  1247 non-null float64
Life_Expectancy_(White)                     1726 non-null float64
#_Deaths                                    3061 non-null float64
Age-Adjusted_Mortality                      3061 non-null float64
Age-Adjusted_Mortality_(Black)              1353 non-null float64
Age-Adjusted_Mortality_(Hispanic)           838 non-null float64
Age-Adjusted_Mortality_(White)              1581 non-null float64
#_Deaths.1                                  1920 non-null float64
Child_Mortality_Rate                    

In [271]:
df.head()

,FIPS,State,County,Life_Expectancy,Life_Expectancy_(Black),Life_Expectancy_(Hispanic),Life_Expectancy_(White),#_Deaths,Age-Adjusted_Mortality,Age-Adjusted_Mortality_(Black),...,#_Hispanic,%_Hispanic,#_Non-Hispanic_White,%_Non-Hispanic_White,#_Not_Proficient_in_English,%_Not_Proficient_in_English,%_Female,#_Rural,%_Rural,LEB
0,1001,Alabama,Autauga,76.330589,74.600628,NaN,76.337219,815.0,438.7,532.977595,...,1586,2.857452,41336,74.473912,430,0.828548,51.342246,22921.0,42.002162,4.140633e+06
1,1003,Alabama,Baldwin,78.599498,75.628919,98.039893,78.537003,2827.0,348.4,511.027189,...,9675,4.550200,176582,83.047388,872,0.454512,51.452772,77060.0,42.279099,1.608083e+07
2,1005,Alabama,Barbour,75.779457,74.240228,NaN,77.568384,451.0,469.8,541.700115,...,1063,4.206569,11613,45.955679,297,1.199273,47.229917,18613.0,67.789635,1.876051e+06
3,1007,Alabama,Bibb,73.928271,72.294686,NaN,74.109510,445.0,563.7,700.414208,...,598,2.638080,16842,74.298571,84,0.394348,46.453150,15663.0,68.352607,1.638776e+06
4,1009,Alabama,Blount,74.597767,NaN,78.577094,74.098247,1050.0,501.8,NaN,...,5549,9.565097,50439,86.944306,1013,1.870004,50.688639,51562.0,89.951502,NaN


### Exploratory Data Analysis

#### Renaming columns

In [272]:
columns_name = df.columns
new_column_names = []
for column_name in columns_name:
    new_column_names.append(column_name.replace(" ", "_"))
pprint(new_column_names[:5])
df.columns = new_column_names

['FIPS', 'State', 'County', 'Life_Expectancy', 'Life_Expectancy_(Black)']


#### Checking number of missing values by columns and dropping useless columns

In [273]:
for column in new_column_names:
    if df[column].isna().sum() == 3194:
        df.drop(column, axis=1, inplace=True)
    elif ("CI_-_Low" in column or "CI_-_High" in column):
        df.drop(column, axis=1, inplace=True)
    elif df[column].isna().sum() < 100:
        pass
#         print(column,df[column].isna().sum(),"\n")
    else:
        continue
        print(column,df[column].isna().sum(),"\n")

#### Delete rows missing Life Expectancy

In [257]:
df = df.dropna(subset=["Life_Expectancy"])

### Group by States

In [279]:
states_df = df.groupby(['State']).mean()

In [259]:
#df.dropna(subset=["Life_Expectancy_(Black)"])["Life_Expectancy_(Black)"]

In [282]:
states_df.columns

Index(['FIPS', 'Life_Expectancy', 'Life_Expectancy_(Black)',
       'Life_Expectancy_(Hispanic)', 'Life_Expectancy_(White)', '#_Deaths',
       'Age-Adjusted_Mortality', 'Age-Adjusted_Mortality_(Black)',
       'Age-Adjusted_Mortality_(Hispanic)', 'Age-Adjusted_Mortality_(White)',
       '#_Deaths.1', 'Child_Mortality_Rate', 'Child_Mortality_Rate_(Black)',
       'Child_Mortality_Rate_(Hispanic)', 'Child_Mortality_Rate_(White)',
       '#_Deaths.2', 'Infant_Mortality_Rate', 'Infant_Mortality_Rate_(Black)',
       'Infant_Mortality_Rate_(Hispanic)', 'Infant_Mortality_Rate_(White)',
       '%_Frequent_Physical_Distress', '%_Frequent_Mental_Distress',
       '%_Diabetic', '#_HIV_Cases', 'HIV_Prevalence_Rate', '#_Food_Insecure',
       '%_Food_Insecure', '#_Limited_Access', '%_Limited_Access',
       '#_Drug_Overdose_Deaths', 'Drug_Overdose_Mortality_Rate',
       '#_Motor_Vehicle_Deaths', 'MV_Mortality_Rate', '%_Insufficient_Sleep',
       '#_Uninsured', '%_Uninsured', '#_Uninsured.1', '%

In [307]:
# Define a lambda function to compute the weighted mean:
wm = lambda x: np.average(x, weights=df.loc[x.index, "Population"])

# Define a dictionary with the functions to apply for a given column:
f = {"Population":"sum",
     "Life_Expectancy": wm, 
#      "Life_Expectancy_(Black)": "median",
#      "Life_Expectancy_(Hispanic)": "median",
#      "Life_Expectancy_(White)": "median",
     "%_Diabetic": wm,
     "#_HIV_Cases": "sum",
     "#_Food_Insecure":"sum",
     "#_Limited_Access":"sum",
     "#_Drug_Overdose_Deaths":"sum",
     "#_Motor_Vehicle_Deaths":"sum",
     "%_Insufficient_Sleep":wm,
     "Household_Income":wm,
     "%_Female":wm
     }

# Groupby and aggregate with your dictionary:
states_df = df.groupby(["State"]).agg(f)

In [308]:
states_df = states_df.reset_index()
states_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 12 columns):
State                     51 non-null object
Population                51 non-null int64
Life_Expectancy           51 non-null float64
%_Diabetic                51 non-null float64
#_HIV_Cases               51 non-null float64
#_Food_Insecure           51 non-null int64
#_Limited_Access          51 non-null float64
#_Drug_Overdose_Deaths    51 non-null float64
#_Motor_Vehicle_Deaths    51 non-null float64
%_Insufficient_Sleep      51 non-null float64
Household_Income          51 non-null float64
%_Female                  51 non-null float64
dtypes: float64(9), int64(2), object(1)
memory usage: 4.9+ KB


In [306]:
states_df["State"] = states_df["State"].astype('category')
states_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 14 columns):
State                         51 non-null category
Population                    51 non-null int64
Life_Expectancy               51 non-null float64
Life_Expectancy_(Black)       51 non-null float64
Life_Expectancy_(Hispanic)    51 non-null float64
Life_Expectancy_(White)       51 non-null float64
%_Diabetic                    51 non-null float64
#_HIV_Cases                   51 non-null float64
#_Food_Insecure               51 non-null int64
#_Limited_Access              51 non-null float64
#_Drug_Overdose_Deaths        51 non-null float64
#_Motor_Vehicle_Deaths        51 non-null float64
%_Insufficient_Sleep          51 non-null float64
Household_Income              51 non-null float64
dtypes: category(1), float64(11), int64(2)
memory usage: 8.2 KB


TODO: Transform every # into percentages

#### Rename Deaths columns

limited access = limited access to healthy foods

In [222]:
states = df.State.unique()